In [1]:
import pandas as pd
import numpy as np

In [2]:
path = '/home/baitong/pywork/cellDetection/sub/'
path2 = '/home/baitong/pywork/cellDetection/sub2/'

In [3]:
%%time
# sub_resnet34_256 = pd.read_csv(path+'protein_classification_c.csv',low_memory=False)##0.453
sub_resnet34_512 = pd.read_csv(path+'protein_classification512.csv',low_memory=False)##0.470

sub_resnet34_all = pd.read_csv(path2+'ResNet34_allprotein_classification_05.csv',low_memory=False)##0.480

sub_resnet50_512 = pd.read_csv(path+'resnet50_512protein_classification_05.csv',low_memory=False)##0.477
sub_wrn_512      = pd.read_csv(path+'wrnprotein_classification_05.csv',low_memory=False)##0.483
sub_resnext50_512= pd.read_csv(path+'resnext50protein_classification.csv',low_memory=False)##0.484
sub_resnet101_512= pd.read_csv(path+'resnet101protein_classification.csv',low_memory=False)##0.486
# weak_ensem = pd.read_csv(path+'weak_ensem.csv',low_memory=False)##0.488

CPU times: user 75 ms, sys: 220 µs, total: 75.2 ms
Wall time: 106 ms


In [4]:
df_list = [sub_resnet34_512,sub_resnet50_512,sub_wrn_512,sub_resnet101_512,sub_resnet34_all,sub_resnext50_512]
# score_list = [0.453,0.470,0.477,0.483,0.484,0.486]
# score_sum = 0
# for s in score_list:
#     score_sum += s

In [5]:
# sub_resnet34_256.head()

In [6]:
name_label_dict = {
 'Nucleoplasm' :0,
 'Nuclear membrane':1,
 'Nucleoli':2,   
 'Nucleoli fibrillar center':3,
 'Nuclear speckles':4,
 'Nuclear bodies':5,
 'Endoplasmic reticulum':6,  
 'Golgi apparatus':7,
 'Peroxisomes':8,
 'Endosomes'   :9,              
  'Lysosomes'  :10,                         
  'Intermediate filaments':11,
  'Actin filaments':12,
  'Focal adhesion sites' :13,  
  'Microtubules':14,
  'Microtubule ends'  :15,
  'Cytokinetic bridge'   :16,
  'Mitotic spindle':17,
  'Microtubule organizing center'  :18,
  'Centrosome':19,
  'Lipid droplets':20,
  'Plasma membrane':21,
  'Cell junctions' :22,
  'Mitochondria':23,
  'Aggresome':24,
  'Cytosol':25,
  'Cytoplasmic bodies' :26,  
  'Rods & rings':27 }
def strToIntArr(str_):
    num_arr = []
    if type(str_) != str:
        num_arr.append(str_)
    else:    
        str_arr = str_.split()
        for s in str_arr:
            num_arr.append(int(s))
    return num_arr
for df in df_list:
    df['Predicted'] = df['Predicted'].apply(strToIntArr)
features = ['Nucleoplasm','Nuclear membrane','Nucleoli',   'Nucleoli fibrillar center',
    'Nuclear speckles','Nuclear bodies','Endoplasmic reticulum','Golgi apparatus','Peroxisomes',
    'Endosomes','Lysosomes','Intermediate filaments','Actin filaments','Focal adhesion sites',   
    'Microtubules','Microtubule ends',  'Cytokinetic bridge', 'Mitotic spindle','Microtubule organizing center',  
     'Centrosome','Lipid droplets','Plasma membrane', 'Cell junctions', 'Mitochondria',
     'Aggresome','Cytosol','Cytoplasmic bodies','Rods & rings']
for df in df_list:
    for col in features:
        df[col] = df['Predicted'].apply(lambda x :1 if name_label_dict[col] in x else 0)
for df in df_list:
    df.fillna(0,inplace=True)       

In [7]:
ensem_df = pd.DataFrame(columns = ['Id','Nucleoplasm','Nuclear membrane','Nucleoli',   'Nucleoli fibrillar center',
    'Nuclear speckles','Nuclear bodies','Endoplasmic reticulum',   'Golgi apparatus','Peroxisomes',
    'Endosomes','Lysosomes','Intermediate filaments','Actin filaments','Focal adhesion sites',   
    'Microtubules','Microtubule ends',  'Cytokinetic bridge', 'Mitotic spindle','Microtubule organizing center',  
     'Centrosome','Lipid droplets','Plasma membrane',  'Cell junctions', 'Mitochondria',
     'Aggresome','Cytosol','Cytoplasmic bodies','Rods & rings','pred'])

In [8]:
ensem_df['Id'] = df_list[0]['Id']

In [9]:
ensem_df.fillna(0,inplace=True)

In [10]:
for col in features:
    for df in df_list:
        ensem_df[col] += df[col]/len(df_list)            

In [11]:
for col in features:
    ensem_df[col] = ensem_df[col].apply(lambda x:1 if x>=0.5 else 0)

In [12]:
ensem_df[features].head()

,Nucleoplasm,Nuclear membrane,Nucleoli,Nucleoli fibrillar center,Nuclear speckles,Nuclear bodies,Endoplasmic reticulum,Golgi apparatus,Peroxisomes,Endosomes,...,Microtubule organizing center,Centrosome,Lipid droplets,Plasma membrane,Cell junctions,Mitochondria,Aggresome,Cytosol,Cytoplasmic bodies,Rods & rings
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [13]:
ensem_df['pred'] = ''
ensem_df['pred'] = ensem_df['pred'].astype('str')
for col in features:
    ensem_df['tmp'] = ensem_df[col].apply(lambda x: ' '+str(name_label_dict[col]) if x==1 else '')    
    ensem_df['tmp'] = ensem_df['tmp'].astype('str')
    ensem_df['pred'] = ensem_df['pred'] +  ensem_df['tmp']

In [14]:
ensem_df['pred'] = ensem_df['pred'].apply(lambda x: x[1:])

In [15]:
ensem_df['pred'].head()

0         2
1         5
2    0 5 25
3      0 25
4        25
Name: pred, dtype: object

In [16]:
ensem_df['Predicted'] = ensem_df['pred']

In [17]:
ensem_df[['Id','Predicted']].to_csv('sub_ensem.csv',index=False)

In [18]:
ensem_df[['Id','Predicted']].head()

,Id,Predicted
0,00008af0-bad0-11e8-b2b8-ac1f6b6435d0,2
1,0000a892-bacf-11e8-b2b8-ac1f6b6435d0,5
2,0006faa6-bac7-11e8-b2b7-ac1f6b6435d0,0 5 25
3,0008baca-bad7-11e8-b2b9-ac1f6b6435d0,0 25
4,000cce7e-bad4-11e8-b2b8-ac1f6b6435d0,25
